In [1]:
from __future__ import absolute_import, division, print_function

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print (device_name)

     |████████████████████████████████| 332.1MB 79kB/s 
     |████████████████████████████████| 3.0MB 45.7MB/s 
     |████████████████████████████████| 419kB 57.2MB/s 
     |████████████████████████████████| 61kB 33.1MB/s 
/device:GPU:0


In [0]:
import matplotlib.pyplot as plt
% matplotlib inline
import numpy as np
import pandas as pd
import time
import random
import math

In [3]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
batch_size = 32
emb_size = 300
hidden_size = 512
lr = 1e-3

In [0]:
train_en_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/train.en.txt' # train set
train_ja_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/train.ja.txt' # train set
dev_en_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/dev.en.txt' # dev set
dev_ja_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/dev.ja.txt' # dev set
test_en_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/test.en.txt' 
test_ja_path = '/content/drive/My Drive/Colab Notebooks/cs11747/data/para/test.ja.txt' 

In [0]:
def read_data(file):
    with open(file, "r") as f:
        data = []
        for line in f:
            words = line.lower().strip()
            words = '<start> ' + words + ' <end>'
            data.append(words)
        return data

In [7]:
train_en = read_data(train_en_path)
train_ja = read_data(train_ja_path)
dev_en = read_data(dev_en_path)
dev_ja = read_data(dev_ja_path)
test_en = read_data(test_en_path)
test_ja = read_data(test_ja_path)
print (len(train_en), len(dev_en), len(test_en))

10000 500 500


In [0]:
tokenizer_en = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>') 
tokenizer_en.fit_on_texts(train_en)
train_seq_en = tokenizer_en.texts_to_sequences(train_en)
dev_seq_en = tokenizer_en.texts_to_sequences(dev_en)
test_seq_en = tokenizer_en.texts_to_sequences(test_en)

In [9]:
w2i_en = tokenizer_en.word_index
i2w_en = dict([(value, key) for (key, value) in w2i_en.items()])
nwords_en = len(w2i_en)
print (nwords_en)

7043


In [0]:
tokenizer_ja = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>') 
tokenizer_ja.fit_on_texts(train_ja)
train_seq_ja = tokenizer_ja.texts_to_sequences(train_ja)
dev_seq_ja = tokenizer_ja.texts_to_sequences(dev_ja)
test_seq_ja = tokenizer_ja.texts_to_sequences(test_ja)

In [11]:
w2i_ja = tokenizer_ja.word_index
i2w_ja = dict([(value, key) for (key, value) in w2i_ja.items()])
nwords_ja = len(w2i_ja)
print (nwords_ja)

8061


In [12]:
w2i_en['<start>'], i2w_ja[2]

(2, '。')

In [0]:
def batch_gen(src, tar):
    i = 0
    while i<len(src):
        if i+batch_size < len(src):
            batch_src = tf.keras.preprocessing.sequence.pad_sequences(src[i:i+batch_size], padding='post')
            batch_tar = tf.keras.preprocessing.sequence.pad_sequences(tar[i:i+batch_size], padding='post')
        else:
            batch_src = tf.keras.preprocessing.sequence.pad_sequences(src[i:], padding='post')
            batch_tar = tf.keras.preprocessing.sequence.pad_sequences(tar[i:], padding='post')
        i += batch_size
        yield batch_src, batch_tar

In [0]:
def shuffle_seq(src, tar):
    c = list(zip(src, tar))
    random.shuffle(c)
    src, tar = zip(*c)
    return src, tar

In [0]:
# encode the source sentence to latent variable, get the KL divergence
class Encoder(tf.keras.Model):
    def __init__(self, nwords, embed_size, hidden_size):
        super(Encoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(nwords+1, embed_size, trainable=True, mask_zero=True)
        self.lstm = tf.keras.layers.LSTM(hidden_size, recurrent_initializer='glorot_uniform')
        self.dense1_mean = tf.keras.layers.Dense(hidden_size, activation='tanh')
        self.dense2_mean = tf.keras.layers.Dense(hidden_size, use_bias=False)
        self.dense1_var = tf.keras.layers.Dense(hidden_size, activation='tanh')
        self.dense2_var = tf.keras.layers.Dense(hidden_size, use_bias=False)
        
    def kl_loss(self, mu, log_var): #[batch, hid]
        kl = 1 + log_var - mu * mu - tf.exp(log_var) #[batch, hid]
        kl = tf.reduce_sum(kl, axis=-1) #[batch]
        return -0.5 * kl
    
    # reparameterization trick: get the latent variable from the mean vector and variance vector
    def reparameterize(self, mu, log_var): #[batch, hid]
        std = tf.exp(log_var * 0.5) #[batch, hid]
        eps = tf.random.normal(tf.shape(mu)) #[batch, hid]
        z = mu + std * eps #[batch, hid]
        return z

    def call(self, x, mask): #[batch, t]
        x = self.embedding(x) #[batch, t, emb]
        # passing all time steps to the lstm, get the last output
        output = self.lstm(x, mask=mask) # [batch, hidden]
        # mean of the latent variable
        mu = self.dense1_mean(output)
        mu = self.dense2_mean(mu) #[batch, hid]
        # log variance of the latent variable
        log_var = self.dense1_var(output)
        log_var = self.dense2_var(log_var) #[batch, hid]
        
        kl = self.kl_loss(mu, log_var) #[batch]
        z = self.reparameterize(mu, log_var) #[batch, hid]

        return kl, z

In [16]:
encoder = Encoder(nwords_en, emb_size, hidden_size)

W0523 00:56:16.095211 139944518780800 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f46ed5177f0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [0]:
# decode to get the target sentences. use latent variable as initial states
class Decoder(tf.keras.Model):
    def __init__(self, nwords, embed_size, hidden_size):
        super(Decoder, self).__init__()
        self.embedding = tf.keras.layers.Embedding(nwords+1, embed_size, trainable=True, mask_zero=True)
        self.lstm = tf.keras.layers.LSTM(hidden_size, return_sequences=True, recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(nwords+1)

    def call(self, x, mask, init_hidden, init_cell): # x:[batch, t], init_hidden:[batch, hid], init_cell:[batch, hid]
        x = self.embedding(x) #[batch, t, emb]
        # passing all time steps to the LSTM
        outputs = self.lstm(x, mask=mask, initial_state=[init_hidden, init_cell]) #[batch, t, hid]
        outputs = self.fc(outputs) #[batch, t, nwords+1]

        return outputs

In [18]:
decoder = Decoder(nwords_ja, emb_size, hidden_size)

W0523 00:56:16.134467 139944518780800 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f46e0241518>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# get the reconstruction loss
def loss_function(real, pred, mask): # real:[batch, t], pred:[batch, t, nwords+1], mask: [batch, t]
    reconstruction_loss = loss_object(real, pred) #[batch, t]

    mask = tf.cast(mask, dtype=reconstruction_loss.dtype)
    reconstruction_loss *= mask #[batch, t]

    return tf.reduce_sum(reconstruction_loss, axis=-1) #[batch]

In [0]:
def train_step(src, tar): # scr, tar: [batch, t]
    loss = 0  
    with tf.GradientTape() as tape:
        mask_src = tf.math.logical_not(tf.math.equal(src, 0)) #[batch, t]
        kl, z = encoder(src, mask_src) # kl: [batch], z: [batch, hid]
        init_hidden = z
        init_cell = tf.tanh(z)
        mask_tar = tf.math.logical_not(tf.math.equal(tar, 0)) #[batch, t]
        logits = decoder(tar, mask_tar, init_hidden, init_cell) #[batch, t, nwords+1]
        mask_loss = tf.math.logical_not(tf.math.logical_or(tf.math.equal(tar, 0), tf.math.equal(tar, w2i_ja['<end>']))) #[batch, t]
        tar_real = tf.concat([tar[:, 1:], tf.zeros([tf.shape(tar)[0], 1])], axis=-1) # [batch, t] shift the target sequence one word to left
        recon = loss_function(tar_real, logits, mask_loss) #[batch]
        loss = tf.reduce_mean(recon + kl)

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
  
    return loss

In [0]:
def eval_step(src, tar):
    loss = 0    
    mask_src = tf.math.logical_not(tf.math.equal(src, 0)) #[batch, t]
    kl, z = encoder(src, mask_src) # kl: [batch], z: [batch, hid]
    init_hidden = z
    init_cell = tf.tanh(z)
    mask_tar = tf.math.logical_not(tf.math.equal(tar, 0)) #[batch, t]
    logits = decoder(tar, mask_tar, init_hidden, init_cell) #[batch, t, nwords+1]
    mask_loss = tf.math.logical_not(tf.math.logical_or(tf.math.equal(tar, 0), tf.math.equal(tar, w2i_ja['<end>']))) #[batch, t]
    tar_real = tf.concat([tar[:, 1:], tf.zeros([tf.shape(tar)[0], 1])], axis=-1) # [batch, t] shift the target sequence one word to left
    recon = loss_function(tar_real, logits, mask_loss) #[batch]
    loss = tf.reduce_mean(recon + kl)
  
    return loss

In [22]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    print ('Epoch {} start ...... '.format(epoch + 1))
    train_seq_en, train_seq_ja = shuffle_seq(train_seq_en, train_seq_ja)
    total_loss = 0
    batch = 0
    for src, tar in batch_gen(train_seq_en, train_seq_ja):
        batch_loss = train_step(src, tar)
        total_loss += batch_loss
        batch += 1

        if batch % 50 == 0:
            print('  ---- Batch {} Loss {:.4f}'.format(batch, batch_loss))
    
    # Evaluate on dev set
    dev_loss = 0
    batch_eval = 0
    for src, tar in batch_gen(dev_seq_en, dev_seq_ja):
        batch_loss = eval_step(src, tar)
        dev_loss += batch_loss
        batch_eval += 1

    print('Epoch {} finished in {} seconds. Training loss {:.4f}. Evaluation loss {:.4f}.'.format(epoch + 1, time.time() - start, total_loss / batch, dev_loss / batch_eval))

Epoch 1 start ...... 
  ---- Batch 50 Loss 58.5305
  ---- Batch 100 Loss 50.5471
  ---- Batch 150 Loss 37.6738
  ---- Batch 200 Loss 44.6816
  ---- Batch 250 Loss 36.2839
  ---- Batch 300 Loss 29.3525
Epoch 1 finished in 187.98851895332336 seconds. Training loss 44.1022. Evaluation loss 30.9588.
Epoch 2 start ...... 
  ---- Batch 50 Loss 27.5488
  ---- Batch 100 Loss 24.3035
  ---- Batch 150 Loss 32.2130
  ---- Batch 200 Loss 38.3843
  ---- Batch 250 Loss 33.1509
  ---- Batch 300 Loss 25.8478
Epoch 2 finished in 183.21207976341248 seconds. Training loss 29.7858. Evaluation loss 27.2449.
Epoch 3 start ...... 
  ---- Batch 50 Loss 27.7245
  ---- Batch 100 Loss 27.7768
  ---- Batch 150 Loss 21.3044
  ---- Batch 200 Loss 28.0052
  ---- Batch 250 Loss 29.6176
  ---- Batch 300 Loss 27.8073
Epoch 3 finished in 184.0655255317688 seconds. Training loss 26.6978. Evaluation loss 25.9557.
Epoch 4 start ...... 
  ---- Batch 50 Loss 31.3521
  ---- Batch 100 Loss 27.5336
  ---- Batch 150 Loss 22.4223